In [2]:
import requests
from bs4 import BeautifulSoup



In [13]:
import json
import re
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    # return jsonl data
    return data

def _get_discussion(soup):
    discussions = soup.find_all('div',class_='relative h-16 w-full overflow-hidden')
    if discussions is None:
        return []
    return discussions

def _get_discussion_href(discussions_list, model_id):
    href_list = []
    for discussion in discussions:

        svg = discussion.find('svg')
        svg_classes = svg['class']
        if 'viewbox' in svg.attrs:
            view_box = svg['viewbox']
        else:
            view_box= None
        # class
        # text-green-600 = commit
        # text-blue-600 = issue data
        # text-purple-600 = closed issue data
        # view box attibute 0 0 32 32
        if ('text-blue-600' in svg_classes or 'text-purple-600' in svg_classes) and view_box=='0 0 32 32' :

            title = discussion.find('h4').text.strip()
            href = discussion.find('a')['href']
            discussion_id = href.split('/')[-1]
            author = discussion.find('span',class_='contents').find('a')['href'][1:]
            href_list.append(
                {"ID":f'{model_id}-{discussion_id}',
                '主題':title,
                '来源':author,
                '回复':[],
                '元数据':{'href':href}
                }
            )
    print('href_list',href_list)
    return href_list

def get_comment(discussion_obj_list):
    for discussion_obj in discussion_obj_list:
        href = discussion_obj['元数据']['href']
        res = requests.get(f"https://huggingface.co{href}")
        soup = BeautifulSoup(res.text,'html.parser')
        conversations = soup.find_all('div',class_=re.compile('scroll-mt-4'))
        
        for i,conversation in enumerate(conversations):
            blockquote = ('\n').join([convs.text for convs in conversation.select('blockquote p')])
            try:
                comment = ('\n').join([convs.text for convs in conversation.select('div > p')])
            except Exception as e:
                comment = conversation.find('span',class_='peer italic text-gray-400').text
            reply_time = conversation.find('time')['datetime']
            try:
                author = conversation.find('div',class_='mr-2 flex flex-shrink-0 items-center').find('a').text
            except Exception as e:
                author = conversation.find('div',class_='mr-2').text
            discussion_obj['回复'].append(
                {
                    "楼ID":str(i),
                    "回复":comment,
                    "扩展字段":{
                        "回复人":author,
                        "引用内容":blockquote,
                        "回复时间":reply_time
                    }
                }
            )
            
    return discussion_obj_list

In [7]:
repo_list = load_jsonl('temp/datasets_20241102.jsonl')
for repo in repo_list:
    repo_id = repo['id']
    res = requests.get(f"https://huggingface.co/datasets/{repo_id}/discussions?status=open")
    soup = BeautifulSoup(res.text,'html.parser')
    discussions = _get_discussion(soup)
    
    res = requests.get(f"https://huggingface.co/datasets/{repo_id}/discussions?status=close")
    soup = BeautifulSoup(res.text,'html.parser')
    discussions += _get_discussion(soup)
    discussion_obj_list = _get_discussion_href(discussions, repo_id)
    conversations = get_comment(discussion_obj_list)

FileNotFoundError: [Errno 2] No such file or directory: 'temp/datasets_20241102.jsonl'

In [19]:

repo_id='neuralwork/arxiver'
res = requests.get(f"https://huggingface.co/datasets/{repo_id}/discussions?status=open")
soup = BeautifulSoup(res.text,'html.parser')
open_discussions = _get_discussion(soup)
res = requests.get(f"https://huggingface.co/datasets/{repo_id}/discussions?status=close")
soup = BeautifulSoup(res.text,'html.parser')
close_discussions = _get_discussion(soup)
print(open_discussions)
print(close_discussions)
discussion_obj_list = _get_discussion_href(discussions, repo_id)
conversations = get_comment(discussion_obj_list)

[]
[]
href_list []


In [18]:
conversations

[]